In [17]:
!pip install scikit-learn

In [18]:
!pip install -r requirements.txt

In [19]:
# Install missing packages
import sys
!{sys.executable} -m pip install openai groq python-dotenv pydantic

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns




In [21]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split,GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier






In [22]:
df = pd.read_csv('Raw/MiniProjectS.csv')
df.head(15)



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


# 2.1 Data Preprocessing Pipeline

Data Cleaning: Handle inconsistencies (e.g., TotalCharges data type issues)

In [23]:
# Data Cleaning: 
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

# Data validation summary 
validation_summary = {
    'TotalCharges_dtype': str(df['TotalCharges'].dtype),
    'Missing_values': int(df['TotalCharges'].isnull().sum()),
    'Total_records': len(df),
    'Min_value': df['TotalCharges'].min(),
    'Max_value': df['TotalCharges'].max(),
    'Mean_value': round(df['TotalCharges'].mean(), 2)
}

validation_df = pd.DataFrame([validation_summary])
display(validation_df)


,TotalCharges_dtype,Missing_values,Total_records,Min_value,Max_value,Mean_value
0,float64,0,7043,18.8,8684.8,2281.92


# Feature Engineering: Create meaningful derived features

Tenure categories (New, Established, Loyal)

In [24]:
def tenure_category(t):
    if t <= 12:
        return 'New'
    elif t <= 48:
        return 'Established'
    else:
        return 'Loyal'

df['TenureCategory'] = df['tenure'].apply(tenure_category)
display(tenure_analysis)



validation_df = pd.DataFrame([feature_summary])
display(validation_df)

NameError: name 'tenure_analysis' is not defined

Service adoption score

In [ ]:
service_cols = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

df['ServiceAdoptionScore'] = df[service_cols].applymap(lambda x: 1 if x == 'Yes' else 0).sum(axis=1)

# Service adoption analysis
service_analysis = {
    'Feature': 'ServiceAdoptionScore',
    'Min_Score': df['ServiceAdoptionScore'].min(),
    'Max_Score': df['ServiceAdoptionScore'].max(),
    'Mean_Score': round(df['ServiceAdoptionScore'].mean(), 2),
    'Total_Services': len(service_cols),
    'High_Adopters_%': round((df['ServiceAdoptionScore'] >= 5).mean() * 100, 1)
}

# Display service adoption summary
service_summary = pd.DataFrame([service_analysis])
display(service_summary)





,Feature,Min_Score,Max_Score,Mean_Score,Total_Services,High_Adopters_%
0,ServiceAdoptionScore,0,8,3.36,9,31.1


Average monthly charges per service

In [ ]:
df['AvgMonthlyPerService'] = df['MonthlyCharges'] / df['ServiceAdoptionScore'].replace(0, 1)

# Analysis of average monthly charges per service
avg_charges_analysis = {
    'Feature': 'AvgMonthlyPerService',
    'Min_Value': round(df['AvgMonthlyPerService'].min(), 2),
    'Max_Value': round(df['AvgMonthlyPerService'].max(), 2), 
    'Mean_Value': round(df['AvgMonthlyPerService'].mean(), 2),
    'Median_Value': round(df['AvgMonthlyPerService'].median(), 2),
    'Std_Dev': round(df['AvgMonthlyPerService'].std(), 2)
}

# Display feature analysis
charges_summary = pd.DataFrame([avg_charges_analysis])
display(charges_summary)


# Value efficiency analysis (customers getting good value vs expensive)
df['ValueEfficiency'] = pd.cut(df['AvgMonthlyPerService'], 
                              bins=[0, 10, 20, 30, float('inf')], 
                              labels=['Excellent_Value', 'Good_Value', 'Fair_Value', 'Expensive'])

value_distribution = df['ValueEfficiency'].value_counts(normalize=True).mul(100).round(1)
value_df = pd.DataFrame({'Value_Category': value_distribution.index, 'Percentage_%': value_distribution.values})
display(value_df)


,Feature,Min_Value,Max_Value,Mean_Value,Median_Value,Std_Dev
0,AvgMonthlyPerService,10.42,72.25,22.51,19.75,11.58


,Value_Category,Percentage_%
0,Good_Value,55.3
1,Fair_Value,31.4
2,Expensive,13.3
3,Excellent_Value,0.0


Payment reliability indicators

In [ ]:
df['PaymentRisk'] = df.apply(lambda row: 1 if row['PaperlessBilling']=='Yes' and row['Contract']=='Month-to-month' else 0, axis=1)

# Payment Risk Analysis
payment_risk_analysis = {
    'Feature': 'PaymentRisk',
    'High_Risk_Count': int(df['PaymentRisk'].sum()),
    'Low_Risk_Count': int((df['PaymentRisk'] == 0).sum()),
    'High_Risk_Percentage_%': round(df['PaymentRisk'].mean() * 100, 1),
    'Total_Customers': len(df)
}

# Display payment risk summary
risk_summary = pd.DataFrame([payment_risk_analysis])
display(risk_summary)

# Churn analysis by payment risk
churn_by_payment_risk = df.groupby('PaymentRisk')['Churn'].apply(
    lambda x: (x == 'Yes').mean() * 100
).round(1)

risk_churn_analysis = pd.DataFrame({
    'Payment_Risk_Level': ['Low Risk', 'High Risk'],
    'Customer_Count': [int((df['PaymentRisk'] == 0).sum()), int(df['PaymentRisk'].sum())],
    'Churn_Rate_%': [churn_by_payment_risk[0], churn_by_payment_risk[1]],
    'Risk_Multiplier': [1.0, round(churn_by_payment_risk[1] / churn_by_payment_risk[0], 1)]
})

display(risk_churn_analysis)

# Cross-analysis
payment_by_tenure = pd.crosstab(df['TenureCategory'], df['PaymentRisk'], normalize='index').mul(100).round(1)
payment_by_tenure.columns = ['Low_Risk_%', 'High_Risk_%']
display(payment_by_tenure)

# Business impact assessment
high_risk_customers = df[df['PaymentRisk'] == 1]
business_impact = {
    'Total_High_Risk_Customers': len(high_risk_customers),
    'High_Risk_Monthly_Revenue': round(high_risk_customers['MonthlyCharges'].sum(), 2),
    'Avg_Monthly_Charge_High_Risk': round(high_risk_customers['MonthlyCharges'].mean(), 2),
    'High_Risk_Churn_Rate_%': round((high_risk_customers['Churn'] == 'Yes').mean() * 100, 1),
    'Revenue_at_Risk': round(high_risk_customers[high_risk_customers['Churn'] == 'Yes']['MonthlyCharges'].sum(), 2)
}

impact_df = pd.DataFrame([business_impact])
display(impact_df)


,Feature,High_Risk_Count,Low_Risk_Count,High_Risk_Percentage_%,Total_Customers
0,PaymentRisk,2586,4457,36.7,7043


,Payment_Risk_Level,Customer_Count,Churn_Rate_%,Risk_Multiplier
0,Low Risk,4457,13.9,1.0
1,High Risk,2586,48.3,3.5


,Low_Risk_%,High_Risk_%
TenureCategory,,
Established,58.6,41.4
Loyal,88.0,12.0
New,43.6,56.4


,Total_High_Risk_Customers,High_Risk_Monthly_Revenue,Avg_Monthly_Charge_High_Risk,High_Risk_Churn_Rate_%,Revenue_at_Risk
0,2586,188010.8,72.7,48.3,95936.45


 Encoding Strategies: Compare Different Encoding Methods for Categorical Variables


In [ ]:
def comprehensive_encoding_analysis(df):
    
    # Test variables for comprehensive analysis
    test_variables = ['Contract', 'PaymentMethod', 'InternetService', 'gender']
    
    # Variables summary being tested
    variables_summary = []
    for var in test_variables:
        if var in df.columns:
            variables_summary.append({
                'Variable': var,
                'Unique_Categories': df[var].nunique(),
                'Most_Common': df[var].mode()[0],
                'Sample_Values': str(list(df[var].unique()[:3]))[1:-1]
            })
    
    variables_df = pd.DataFrame(variables_summary)
    
    # Detailed encoding methods comparison
    encoding_methods_analysis = []
    sample_transformations = {}
    
    for var in test_variables:
        if var in df.columns:
            # Original data
            original_data = df[var]
            n_categories = original_data.nunique()
            
            # Method 1: Label Encoding
            le = LabelEncoder()
            label_encoded = le.fit_transform(original_data)
            
            # Method 2: One-Hot Encoding  
            onehot_df = pd.get_dummies(original_data, prefix=var)
            
            # Method 3: Target Encoding
            target_means = df.groupby(var)['Churn'].apply(lambda x: (x == 'Yes').mean())
            target_encoded = original_data.map(target_means)
            
            # Method 4: Frequency Encoding
            freq_encoding = original_data.map(original_data.value_counts())
            
            # Store sample transformations for display
            sample_df = pd.DataFrame({
                'Original': original_data.head(6),
                'Label_Encoded': label_encoded[:6],
                'Target_Encoded': target_encoded.head(6).round(4),
                'Frequency_Encoded': freq_encoding.head(6)
            })
            
            # Add one-hot columns to sample
            for col in onehot_df.columns[:3]:  # Show first 3 onehot columns
                sample_df[f'OneHot_{col}'] = onehot_df[col].head(6).astype(int)
            
            sample_transformations[var] = sample_df
            
            # Comprehensive encoding analysis
            encoding_methods_analysis.append({
                'Variable': var,
                'Categories': n_categories,
                'Label_Range': f'0-{n_categories-1}',
                'OneHot_Features': onehot_df.shape[1],
                'Target_Range': f'{target_encoded.min():.3f}-{target_encoded.max():.3f}',
                'Memory_Original_KB': round(original_data.memory_usage(deep=True) / 1024, 2),
                'Memory_OneHot_KB': round(onehot_df.memory_usage(deep=True).sum() / 1024, 2),
                'Sparsity_OneHot': round((onehot_df == 0).sum().sum() / onehot_df.size, 3),
                'Best_For_TreeModels': 'Target_Encoding',
                'Best_For_LinearModels': 'OneHot_Encoding' if n_categories <= 10 else 'Target_Encoding'
            })
    
    return pd.DataFrame(encoding_methods_analysis), sample_transformations, variables_df

# Run comprehensive encoding analysis
encoding_analysis, transformations_samples, tested_vars = comprehensive_encoding_analysis(df)

# Display results
display(tested_vars)
display(encoding_analysis)


,Variable,Unique_Categories,Most_Common,Sample_Values
0,Contract,3,Month-to-month,"'Month-to-month', 'One year', 'Two year'"
1,PaymentMethod,4,Electronic check,"'Electronic check', 'Mailed check', 'Bank tran..."
2,InternetService,3,Fiber optic,"'DSL', 'Fiber optic', 'No'"
3,gender,2,Male,"'Female', 'Male'"


,Variable,Categories,Label_Range,OneHot_Features,Target_Range,Memory_Original_KB,Memory_OneHot_KB,Sparsity_OneHot,Best_For_TreeModels,Best_For_LinearModels
0,Contract,3,0-2,3,0.028-0.427,414.88,20.76,0.667,Target_Encoding,OneHot_Encoding
1,PaymentMethod,4,0-3,4,0.152-0.453,464.87,27.64,0.750,Target_Encoding,OneHot_Encoding
2,InternetService,3,0-2,3,0.074-0.419,380.48,20.76,0.667,Target_Encoding,OneHot_Encoding
3,gender,2,0-1,2,0.262-0.269,371.47,13.88,0.500,Target_Encoding,OneHot_Encoding


Feature Scaling: Apply appropriate scaling for numerical features

In [ ]:

def comprehensive_feature_scaling_analysis(df):
  
    # Define numerical columns 
    numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'ServiceAdoptionScore', 'AvgMonthlyPerService']
    
    # Verify all columns exist
    available_cols = [col for col in numerical_cols if col in df.columns]
    
    # Before scaling analysis
    before_scaling_stats = []
    for col in available_cols:
        stats = {
            'Variable': col,
            'Mean_Before': round(df[col].mean(), 4),
            'Std_Before': round(df[col].std(), 4),
            'Min_Before': round(df[col].min(), 4),
            'Max_Before': round(df[col].max(), 4),
            'Range_Before': round(df[col].max() - df[col].min(), 4),
            'Skewness_Before': round(df[col].skew(), 4)
        }
        before_scaling_stats.append(stats)
    
    before_stats_df = pd.DataFrame(before_scaling_stats)
    
    # Apply different scaling methods
    scaling_results = {}
    
    # 1. StandardScaler (Z-score )
    standard_scaler = StandardScaler()
    df_standard = df.copy()
    df_standard[available_cols] = standard_scaler.fit_transform(df[available_cols])
    scaling_results['StandardScaler'] = df_standard[available_cols]
    
    # 2. MinMaxScaler (0-1 normalization)
    minmax_scaler = MinMaxScaler()
    df_minmax = df.copy()
    df_minmax[available_cols] = minmax_scaler.fit_transform(df[available_cols])
    scaling_results['MinMaxScaler'] = df_minmax[available_cols]
    
    # 3. RobustScaler (median-based scaling)
    robust_scaler = RobustScaler()
    df_robust = df.copy()
    df_robust[available_cols] = robust_scaler.fit_transform(df[available_cols])
    scaling_results['RobustScaler'] = df_robust[available_cols]
    
    # After scaling analysis
    after_scaling_analysis = []
    
    for scaler_name, scaled_data in scaling_results.items():
        for col in available_cols:
            analysis = {
                'Scaler_Method': scaler_name,
                'Variable': col,
                'Mean_After': round(scaled_data[col].mean(), 4),
                'Std_After': round(scaled_data[col].std(), 4),
                'Min_After': round(scaled_data[col].min(), 4),
                'Max_After': round(scaled_data[col].max(), 4),
                'Range_After': round(scaled_data[col].max() - scaled_data[col].min(), 4)
            }
            after_scaling_analysis.append(analysis)
    
    after_stats_df = pd.DataFrame(after_scaling_analysis)
    
    return before_stats_df, after_stats_df, scaling_results, standard_scaler

# Run comprehensive scaling analysis
before_stats, after_stats, scaling_methods, fitted_scaler = comprehensive_feature_scaling_analysis(df)

# Display before scaling statistics
display(before_stats)


,Variable,Mean_Before,Std_Before,Min_Before,Max_Before,Range_Before,Skewness_Before
0,tenure,32.3711,24.5595,0.0000,72.00,72.0000,0.2395
1,MonthlyCharges,64.7617,30.0900,18.2500,118.75,100.5000,-0.2205
2,TotalCharges,2281.9169,2265.2704,18.8000,8684.80,8666.0000,0.9638
3,ServiceAdoptionScore,3.3629,2.0620,0.0000,8.00,8.0000,0.4508
4,AvgMonthlyPerService,22.5100,11.5762,10.4167,72.25,61.8333,2.4927


# 2.2 Ensemble Model Implementation

Implement and compare the following ensemble methods:

In [ ]:

X = df.drop(columns=['customerID','Churn'])
y = df['Churn'].map({'No':0,'Yes':1})  # binary
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [ ]:

def preprocess_and_split(df):
  
    
    # --- Feature Engineering: Tenure Categories ---
    def tenure_category(tenure):
        if tenure < 12:
            return "New"
        elif tenure < 36:
            return "Established"
        else:
            return "Loyal"
    
    if 'tenure' in df.columns:
        df['TenureCategory'] = df['tenure'].apply(tenure_category)
    
    # --- Feature / Target Separation ---
    X = df.drop(columns=['customerID', 'Churn'])
    y = df['Churn'].map({'No': 0, 'Yes': 1})  # binary target
    
    #  Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        stratify=y,
        test_size=0.2,
        random_state=42
    )
    
    return X_train, X_test, y_train, y_test

# Use the funtion to spliit data
X_train, X_test, y_train, y_test = preprocess_and_split(df)


2.2.1 Bagging Method: Random Forest

Implementation: Use scikit-learn RandomForestClassifier

In [ ]:


#  Encode categorical variables 
X_encoded = pd.get_dummies(X, drop_first=True)

#  Split again with encoded features 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, stratify=y, test_size=0.2, random_state=42
)

#  Train RandomForest 
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    min_samples_split=5,
    random_state=42
)
rf.fit(X_train, y_train)

#  Predictions & Evaluation 
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

#Feature importance 
importances = pd.Series(rf.feature_importances_, index=X_train.columns)\
                .sort_values(ascending=False)
print(importances.head(10))


              precision    recall  f1-score   support

           0       0.83      0.90      0.87      1035
           1       0.66      0.51      0.57       374

    accuracy                           0.80      1409
   macro avg       0.75      0.70      0.72      1409
weighted avg       0.79      0.80      0.79      1409

tenure                            0.123650
AvgMonthlyPerService              0.113469
PaymentRisk                       0.108405
TotalCharges                      0.086991
MonthlyCharges                    0.080526
InternetService_Fiber optic       0.063812
TenureCategory_New                0.060164
PaymentMethod_Electronic check    0.054678
Contract_Two year                 0.035726
ValueEfficiency_Good_Value        0.025987
dtype: float64


Hyperparameters to tune

In [ ]:


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [6, 8, 10],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42)
grid = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

best_rf = grid.best_estimator_
best_params = grid.best_params_
best_score = grid.best_score_

best_params, best_score



({'max_depth': 8, 'min_samples_split': 5, 'n_estimators': 100},
 np.float64(0.8028025172537159))

Feature Importance Analysis

In [ ]:


top_features = importances.head(10)
display(top_features)  # works in Jupyter/Colab




tenure                            0.123650
AvgMonthlyPerService              0.113469
PaymentRisk                       0.108405
TotalCharges                      0.086991
MonthlyCharges                    0.080526
InternetService_Fiber optic       0.063812
TenureCategory_New                0.060164
PaymentMethod_Electronic check    0.054678
Contract_Two year                 0.035726
ValueEfficiency_Good_Value        0.025987
dtype: float64

 Business Insights 

Tenure: Longer-term consumers are less likely to leave, while new customers (less than a year) are more vulnerable.

 Average Monthly Per Service: greater turnover is correlated with greater per-service expenses, indicating price sensitivity.

 PaymentRisk: Customers who make late or irregular payments are more likely to churn; reminding them or offering them alternative payment options may help lower attrition.

 InternetService_Fiber optic: Due to competition or service problems, fiber optic consumers have a slightly higher churn rate and may require retention offers.

 PaymentMethod_Electronic check: High churn signal; these clients can experience security issues or payment friction.

Boosting Methods (XGBoost)

In [ ]:

# Train XGBoost
xgb = XGBClassifier(
    learning_rate=0.1,  # step size for learning
    max_depth=5,        # how deep each tree can be
    n_estimators=200,   # number of trees
    subsample=0.8,      # fraction of data used for each tree
    random_state=42,
    eval_metric='logloss'
)
xgb.fit(X_train, y_train)

# Predictions
y_pred_xgb = xgb.predict(X_test)
print(classification_report(y_test, y_pred_xgb))


              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1035
           1       0.64      0.53      0.58       374

    accuracy                           0.80      1409
   macro avg       0.74      0.71      0.72      1409
weighted avg       0.79      0.80      0.79      1409



In [ ]:

#  Separate features 
X = df.drop(columns=['customerID','Churn'])
y = df['Churn'].map({'No':0,'Yes':1})  # Binary target

#  Identify categorical features 
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

#  Preprocessing 
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # numeric features remain unchanged
)

#  Define XGBoost model 
xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42
)

#  Pipeline 
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', xgb)
])

#  Hyperparameter grid to tune 
param_grid = {
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'classifier__max_depth': [3, 5, 7, 9],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__subsample': [0.6, 0.8, 1.0]
}

#  Train  
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

#  Grid search with 5-fold Cv
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',   # balances precision & recall
    cv=5,
    verbose=1,
    n_jobs=-1
)

# --- Fit the grid search ---
grid_search.fit(X_train, y_train)

# --- Best model & evaluation ---
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)

print("Best Hyperparameters:", grid_search.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred))




Fitting 5 folds for each of 144 candidates, totalling 720 fits


Best Hyperparameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100, 'classifier__subsample': 0.8}

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.91      0.88      1035
           1       0.68      0.53      0.60       374

    accuracy                           0.81      1409
   macro avg       0.76      0.72      0.74      1409
weighted avg       0.80      0.81      0.80      1409



Analysis: Feature importance and model interpretation

Tenure: Early-stage retention techniques are crucial, as evidenced by the much higher churn risk among customers with shorter tenure.

 MonthlyCharges: Higher monthly expenses were linked to higher customer attrition, suggesting that consumers are price sensitive.

 Contract Type: Compared to annual commitments, month-to-month contracts significantly increased churn.

 PaymentMethod_Electronic check: Consumers who used electronic checks had greater churn rates, which may have been related to demographics or the ease of payments.

 InternetService_Fiber optic: Customers have a somewhat greater churn rate than DSL users, which may indicate competitive alternatives or high-quality service.

Gradient boosting is employed by the XGBoost model to award relevance scores according to the frequency and efficiency of each feature's employment in splitting decision trees.  This makes it possible to record non-linear correlations and interaction effects. For instance, clients who have short tenure and high monthly expenses are more likely to churn.

 The model interpretation identifies three crucial areas for intervention from a commercial standpoint:

 Focus on retention: Within the first year, provide clients with early loyalty rewards.

 Price changes: Take into account customized discounts for customers of expensive plans.

 Enhancements to service quality: Look into and resolve fiber optic customers' complaints.

Advanced Boosting: CatBoost

In [ ]:

cat_features = X.select_dtypes(include=['object','category']).columns.tolist()

cat = CatBoostClassifier(
    depth=6,
    learning_rate=0.1,
    iterations=300,
    cat_features=cat_features,
    verbose=0,
    random_state=42
)
cat.fit(X_train, y_train)
y_pred_cat = cat.predict(X_test)
print(classification_report(y_test, y_pred_cat))


              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1035
           1       0.66      0.53      0.59       374

    accuracy                           0.80      1409
   macro avg       0.75      0.72      0.73      1409
weighted avg       0.79      0.80      0.80      1409



Advantages: Automatic categorical encoding, reduced overfitting

In [ ]:

advantages_demo = [
    "Native handling of categorical variables without manual encoding.",
    "Reduced overfitting through ordered boosting technique.",
    "Fast training and inference on large datasets.",
    "Robust to noisy features and missing values."
]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import xgboost as xgb
import time

# Baseline Comparison

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Load example dataset (replace with your actual df)
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Set target column name
target_column = 'target'

# Split into features and target
X = df.drop(columns=[target_column])
y = df[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Logistic Regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)
print(" Logistic Regression ")
print(classification_report(y_test, logreg_pred))

# Decision Tree
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
print(" Decision Tree ")
print(classification_report(y_test, dt_pred))


 Logistic Regression 
              precision    recall  f1-score   support

           0       0.95      0.93      0.94        42
           1       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

 Decision Tree 
              precision    recall  f1-score   support

           0       0.87      0.93      0.90        42
           1       0.96      0.92      0.94        72

    accuracy                           0.92       114
   macro avg       0.91      0.92      0.92       114
weighted avg       0.92      0.92      0.92       114



/Users/kavishkathilakarathna/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Pipeline Construction

Modular Pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Identify categorical and numerical columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessor with scaling for numerical columns and one-hot encoding 
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Random Forest pipeline
rf_pipeline = Pipeline([
    ('prep', preprocessor),
    ('model', RandomForestClassifier(random_state=42))
])

# Fit the pipeline
rf_pipeline.fit(X_train, y_train)

# Predict
y_pred = rf_pipeline.predict(X_test)

# Evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.95      0.93      0.94        42
           1       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



 Cross-validation & Hyperparameter tuning

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

params = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [5, 8, None]
}

grid = GridSearchCV(rf_pipeline, param_grid=params, cv=cv, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)


Best params: {'model__max_depth': 8, 'model__n_estimators': 100}


In [ ]:
df.to_csv('Processed/MiniProjectS_Model_Pipeline.csv', index=False)